<a href="https://colab.research.google.com/github/Mushtatoes/proj/blob/master/Pokemon_Type_1_to_Highest_Base_Stat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pokemon - Base Stats and Types
**Description:** Is there a correlation between a pokemon's primary type and its highest stat? 


**Goal:** Find a pattern in the minds of pokemon designer and learn more about pandas and matplotlib.
<center>
<img src='https://cdn.clipart.email/cb04f2d0395acc25e2abe9da2b289a5a_pikachu-transparent-background-png-mart_1254-1254.png' alt='Pikachu' height='300'> 
</center>

For example, rock pokemon might have defense as their highest stat. Or maybe Psychic pokemon might have generally high special attack. 

Are there any outlier pokemon that may be a bit different from others of the same type?




In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import re

## 1. Import all unique standard Pokemon:
- No mega-evolutions, alternate forms, etc.
- Pulling from PokeAPI
- Manually parse and add data into Dataframe without using read_json
- At time of writing, there are currently 807 unique Pokemon in the API (Up to Gen 7)

In [0]:
pokemon_json = requests.get('https://pokeapi.co/api/v2/pokemon?&limit=5000').json()
# print(pokemon_json)

In [28]:
name_url_raw = pokemon_json['results']

# Filter out non-standard Pokemon using regex on url:
def pokemon_filter(pokemon_object):
  pokemon_search = re.search('^https://pokeapi\.co/api/v2/pokemon/(\d+)/$', pokemon_object['url'])
  if not pokemon_search:
    return False
  return int(pokemon_search.group(1)) <= 10000

name_url = list(filter(pokemon_filter, name_url_raw))

urls = [x['url'] for x in name_url]

pokemon_names = [x['name'] for x in name_url]

t = []
for u in urls:
  json = requests.get(u).json()
  t.append(json['name'])


print(t)

['bulbasaur', 'ivysaur', 'venusaur', 'charmander', 'charmeleon', 'charizard', 'squirtle', 'wartortle', 'blastoise', 'caterpie', 'metapod', 'butterfree', 'weedle', 'kakuna', 'beedrill', 'pidgey', 'pidgeotto', 'pidgeot', 'rattata', 'raticate', 'spearow', 'fearow', 'ekans', 'arbok', 'pikachu', 'raichu', 'sandshrew', 'sandslash', 'nidoran-f', 'nidorina', 'nidoqueen', 'nidoran-m', 'nidorino', 'nidoking', 'clefairy', 'clefable', 'vulpix', 'ninetales', 'jigglypuff', 'wigglytuff', 'zubat', 'golbat', 'oddish', 'gloom', 'vileplume', 'paras', 'parasect', 'venonat', 'venomoth', 'diglett', 'dugtrio', 'meowth', 'persian', 'psyduck', 'golduck', 'mankey', 'primeape', 'growlithe', 'arcanine', 'poliwag', 'poliwhirl', 'poliwrath', 'abra', 'kadabra', 'alakazam', 'machop', 'machoke', 'machamp', 'bellsprout', 'weepinbell', 'victreebel', 'tentacool', 'tentacruel', 'geodude', 'graveler', 'golem', 'ponyta', 'rapidash', 'slowpoke', 'slowbro', 'magnemite', 'magneton', 'farfetchd', 'doduo', 'dodrio', 'seel', 'dew

## 2. Pull names of types and stats to push into DataFrame